# Titanic data analysis

In [583]:
import numpy as np
import pandas as pd

#
training = pd.read_csv('titanic_training.csv', sep=",")

## Display beginning of dataset

In [584]:

training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Display end of dataset

In [585]:
training.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


## Correlations

In [586]:
cor = training.corr()
print(cor)

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.035144  0.036847 -0.057527 -0.001652   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.035144 -0.338481  1.000000 -0.369226  0.083081  0.018443   
Age             0.036847 -0.077221 -0.369226  1.000000 -0.308247 -0.189119   
SibSp          -0.057527 -0.035322  0.083081 -0.308247  1.000000  0.414838   
Parch          -0.001652  0.081629  0.018443 -0.189119  0.414838  1.000000   
Fare            0.012658  0.257307 -0.549500  0.096067  0.159651  0.216225   

                 Fare  
PassengerId  0.012658  
Survived     0.257307  
Pclass      -0.549500  
Age          0.096067  
SibSp        0.159651  
Parch        0.216225  
Fare         1.000000  


## Missing and Empty values

In [587]:
def get_missing(df):
    percent_missing = df.isnull().sum() / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', inplace=True)
    return missing_value_df


print(get_missing(training))

             column_name  percent_missing
PassengerId  PassengerId         0.000000
Survived        Survived         0.000000
Pclass            Pclass         0.000000
Name                Name         0.000000
Sex                  Sex         0.000000
SibSp              SibSp         0.000000
Parch              Parch         0.000000
Ticket            Ticket         0.000000
Fare                Fare         0.000000
Embarked        Embarked         0.002245
Age                  Age         0.198653
Cabin              Cabin         0.771044


## Replace and Cleanup Dataset

In [588]:
# Map male to 0
training.loc[training['Sex'] == 'male', 'Sex'] = 0
# Map female to 1
training.loc[training['Sex'] == 'female', 'Sex'] = 1

# Map the port to S = 1 C = 2 Q = 3
training.loc[training['Embarked'] == 'S', 'Embarked'] = 1
training.loc[training['Embarked'] == 'C', 'Embarked'] = 2
training.loc[training['Embarked'] == 'Q', 'Embarked'] = 3

training.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.00,NaN,1
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.00,B42,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.45,NaN,1
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.00,C148,2
890,891,0,3,"Dooley, Mr. Patrick",0,32.0,0,0,370376,7.75,NaN,3


## Get all name titles

In [589]:
available_titles = training['Name'].str.split(', ').str[1]
available_titles = available_titles.str.split('.').str[0]
training["Title"] = available_titles
distinct_title = list(set(available_titles))

groupedTitles = {
    "married-female": ["Mrs", "Mme"],
    "not-married-female": ["Ms", "Miss", "Mlle"],
    "army": ["Col", "Major", "Capt"],
    "noble": ["Sir", "Master", "Lady", "Jonkheer", "the Countess", "Don"],
    "academic": ["Dr"],
    "chaplain": ["Rev"],
    "not-specified-male": ["Mr"]
}

count = 0
for category, values_to_replace in groupedTitles.items():
    training.loc[training['Title'].isin(values_to_replace), 'Title'] = count
    count = count + 1
print(training['Title'])
print(distinct_title)
print(len(distinct_title))
print(get_missing(training))

#training.head()
#print(training.astype({'Title': 'int64'}).dtypes)

0      6
1      0
2      1
3      0
4      6
      ..
886    5
887    1
888    1
889    6
890    6
Name: Title, Length: 891, dtype: object
['Don', 'Dr', 'Jonkheer', 'Mr', 'Master', 'Mme', 'Rev', 'Major', 'Sir', 'Ms', 'Col', 'Mlle', 'Lady', 'Capt', 'the Countess', 'Mrs', 'Miss']
17
             column_name  percent_missing
PassengerId  PassengerId         0.000000
Survived        Survived         0.000000
Pclass            Pclass         0.000000
Name                Name         0.000000
Sex                  Sex         0.000000
SibSp              SibSp         0.000000
Parch              Parch         0.000000
Ticket            Ticket         0.000000
Fare                Fare         0.000000
Title              Title         0.000000
Embarked        Embarked         0.002245
Age                  Age         0.198653
Cabin              Cabin         0.771044


# Summary Statistics

## Location parameters

In [590]:
import pandas as pd

numeric_columns = ['Age', 'SibSp', 'Parch', 'Fare']
stats = {}
for item in numeric_columns:
    stats[item] = {
        "mean": training[item].mean(),
        "median": training[item].median(),
        "variance": training[item].var(),
        "min": training[item].min(),
        "max": training[item].max(),
    }

print(stats)

{'Age': {'mean': 29.69911764705882, 'median': 28.0, 'variance': 211.0191247463081, 'min': 0.42, 'max': 80.0}, 'SibSp': {'mean': 0.5230078563411896, 'median': 0.0, 'variance': 1.2160430774662894, 'min': 0, 'max': 8}, 'Parch': {'mean': 0.38159371492704824, 'median': 0.0, 'variance': 0.6497282437357467, 'min': 0, 'max': 6}, 'Fare': {'mean': 32.2042079685746, 'median': 14.4542, 'variance': 2469.436845743117, 'min': 0.0, 'max': 512.3292}}


## Count titles

In [591]:
grouped = training[["Title", "PassengerId"]]
grouped = grouped.groupby("Title").agg(['count'])
grouped

,PassengerId
,count
Title,
0,126
1,185
2,5
3,45
4,7
5,6
6,517


## Datensatz vorbereiten

In [592]:
# Datensatz ausdünnen
new_training = training.drop(['Embarked', 'PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare', 'Age'], axis=1)
# Inputs
x_training = new_training.drop('Survived', axis=1)
# Outputs
y_training = new_training['Survived']

dim = len(x_training.columns)

## Print Training Dataset

In [593]:
x_training.head()
y_training.head()

,Pclass,Sex,SibSp,Parch,Title
0,3,0,1,0,6
1,1,1,1,0,0
2,3,1,0,0,1
3,1,1,1,0,0
4,3,0,0,0,6


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

## Scaling

In [594]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_training = sc.fit_transform(x_training)

## Neurales Netz

In [595]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow import sigmoid
from tensorflow.keras import activations

# layers = Schichten im NN-Modell
# Dense = Neuronen im Modell
# Activation = Aktivierungregel (wie sigmoid)
# Dropout = overfitting verhindern

#Netz definieren
model = Sequential()
model.add(Dense(7, activation='relu', input_dim=dim))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train NN

In [596]:
#Train
train = model.fit(x_training, y_training, batch_size=32, epochs=250, validation_split=0.2)
mean_val = np.mean(train.history['val_accuracy'])
max_val = np.max(train.history['val_accuracy'])
print(mean_val)
print(max_val)

Epoch 1/250
23/23 [==============================] - 0s 5ms/step - loss: 0.7975 - accuracy: 0.5365 - val_loss: 0.7427 - val_accuracy: 0.5307
Epoch 2/250
23/23 [==============================] - 0s 1ms/step - loss: 0.7343 - accuracy: 0.5871 - val_loss: 0.7087 - val_accuracy: 0.5698
Epoch 3/250
23/23 [==============================] - 0s 1ms/step - loss: 0.7002 - accuracy: 0.6236 - val_loss: 0.6795 - val_accuracy: 0.6145
Epoch 4/250
23/23 [==============================] - 0s 1ms/step - loss: 0.7012 - accuracy: 0.5899 - val_loss: 0.6610 - val_accuracy: 0.7263
Epoch 5/250
23/23 [==============================] - 0s 1ms/step - loss: 0.6678 - accuracy: 0.6615 - val_loss: 0.6463 - val_accuracy: 0.7486
Epoch 6/250
23/23 [==============================] - 0s 1ms/step - loss: 0.6581 - accuracy: 0.6601 - val_loss: 0.6333 - val_accuracy: 0.7486
Epoch 7/250
23/23 [==============================] - 0s 1ms/step - loss: 0.6424 - accuracy: 0.6952 - val_loss: 0.6206 - val_accuracy: 0.7654
Epoch 8/250
2

## Testpreperation

In [597]:
test = pd.read_csv('titanic_test.csv', sep=",")

# Map male to 0
test.loc[test['Sex'] == 'male', 'Sex'] = 0
# Map female to 1
test.loc[test['Sex'] == 'female', 'Sex'] = 1

# Map the port to S = 1 C = 2 Q = 3
test.loc[test['Embarked'] == 'S', 'Embarked'] = 1
test.loc[test['Embarked'] == 'C', 'Embarked'] = 2
test.loc[test['Embarked'] == 'Q', 'Embarked'] = 3

test["Title"] = available_titles

count = 0
for category, values_to_replace in groupedTitles.items():
    test.loc[test['Title'].isin(values_to_replace), 'Title'] = count
    count = count + 1

# Datensatz ausdünnen
new_test = test.drop(['Embarked', 'PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare', 'Age'], axis=1)
x_test = sc.fit_transform(new_test)

new_test.head()

,Pclass,Sex,SibSp,Parch,Title
0,3,0,0,0,6
1,3,1,1,0,0
2,2,0,0,0,1
3,3,0,0,0,0
4,3,1,1,1,6


## Prediction

In [598]:
prediction = model.predict(x_test).tolist()

se = pd.Series(prediction)

pred = []
raw_pred = []
for val in se:
    if val[0] >= 0.5:
        pred.append(1)
        raw_pred.append(val[0])
    else:
        pred.append(0)
        raw_pred.append(val[0])

test_show = pd.read_csv('titanic_test.csv', sep=",")
test_show['raw_pred'] = raw_pred
test_show['survival_prediction'] = pred

test_show.to_csv('prediction.csv')

In [599]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, './output')

In [606]:
testarray = sc.fit_transform(new_test)
print(testarray)
prediction = model.predict(testarray).tolist()



[[ 0.87348191 -0.75592895 -0.49947002 -0.4002477   0.87418602]
 [ 0.87348191  1.32287566  0.61699237 -0.4002477  -1.45821669]
 [-0.31581919 -0.75592895 -0.49947002 -0.4002477  -1.0694829 ]
 ...
 [ 0.87348191 -0.75592895 -0.49947002 -0.4002477  -1.45821669]
 [ 0.87348191 -0.75592895 -0.49947002 -0.4002477  -1.45821669]
 [ 0.87348191 -0.75592895  0.61699237  0.61989583 -1.0694829 ]]


In [607]:
import pandas as pd

numeric_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Title']
stats = {}
for item in numeric_columns:
    stats[item] = {
        "mean": new_test[item].mean(),
        "median": new_test[item].median(),
        "variance": new_test[item].var(),
        "min": new_test[item].min(),
        "max": new_test[item].max(),
        "stand": new_test[item].std()
    }

print(stats)

{'Pclass': {'mean': 2.2655502392344498, 'median': 3.0, 'variance': 0.7086904638968277, 'min': 1, 'max': 3, 'stand': 0.8418375519640519}, 'Sex': {'mean': 0.36363636363636365, 'median': 0.0, 'variance': 0.23195988663614564, 'min': 0, 'max': 1, 'stand': 0.4816221409322309}, 'SibSp': {'mean': 0.4473684210526316, 'median': 0.0, 'variance': 0.8041777104632064, 'min': 0, 'max': 8, 'stand': 0.8967595611217125}, 'Parch': {'mean': 0.3923444976076555, 'median': 0.0, 'variance': 0.9632026436267241, 'min': 0, 'max': 9, 'stand': 0.9814288785371684}, 'Title': {'mean': 3.751196172248804, 'median': 6.0, 'variance': 6.633391851112412, 'min': 0, 'max': 6, 'stand': 2.5755371966081975}}
